<a href="https://colab.research.google.com/github/Antoine723/ProjetNLP/blob/main/ProjetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf

from gdrive.MyDrive.ProjetNLP.rnn_utils import *

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
data=np.array([df.iloc[0:5][5],df.iloc[800000:800005][5]])

print(data)

[["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
  "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
  '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
  'my whole body feels itchy and like its on fire '
  "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "]
 ['I LOVE @Health4UandPets u guys r the best!! '
  'im meeting up with one of my besties tonight! Cant wait!!  - GIRL TALK!!'
  '@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart. '
  'Being sick can be really cheap when it hurts too much to eat real food  Plus, your friends make you soup'
  '@LovesBrooklyn2 he has that effect on everyone ']]


# Traitement

In [ ]:
def processing(data):

  #MINUSCULES
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=data[i][j].lower()
  # print(data)

  #TOKENIZATION
  tokenizer = nltk.RegexpTokenizer(r'\w+')

  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=tokenizer.tokenize(data[i][j])
  # print(data)

  #STOPWORDS
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[w for w in data[i][j] if not w in list(nltk.corpus.stopwords.words('english'))]
      
  # print(data)

  #STEMMING
  stemmer=LancasterStemmer()
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[stemmer.stem(w) for w in data[i][j]]
  # print(data)

  #LEMMATIZATION
  Word_Lemmatizer = WordNetLemmatizer()
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[Word_Lemmatizer.lemmatize(w) for w in data[i][j]]
  print(data)

  return data

In [ ]:
datas=processing(data)

[[list(['switchfoot', 'http', 'twitp', 'com', '2y1zl', 'awww', 'bum', 'should', 'got', 'david', 'car', 'third', 'day'])
  list(['upset', 'upd', 'facebook', 'text', 'might', 'cry', 'result', 'school', 'today', 'also', 'blah'])
  list(['kenich', 'div', 'many', 'tim', 'bal', 'man', 'sav', '50', 'rest', 'go', 'bound'])
  list(['whol', 'body', 'feel', 'itchy', 'lik', 'fir'])
  list(['nationwideclass', 'behav', 'mad', 'see'])]
 [list(['lov', 'health4uandpets', 'u', 'guy', 'r', 'best'])
  list(['im', 'meet', 'on', 'besty', 'tonight', 'cant', 'wait', 'girl', 'talk'])
  list(['darealsunisakim', 'thank', 'twit', 'ad', 'sunis', 'got', 'meet', 'hin', 'show', 'dc', 'are', 'sweetheart'])
  list(['sick', 'real', 'cheap', 'hurt', 'much', 'eat', 'real', 'food', 'plu', 'friend', 'mak', 'soup'])
  list(['lovesbrooklyn2', 'effect', 'everyon'])]]


In [ ]:
from collections import defaultdict
v=set()
for i in range(len(data)):
  for j in range(len(data[i])):
    for h in range(len(data[i][j])):
      v.add(data[i][j][h])

vocab = defaultdict(lambda: 0)
vocab['<PAD>'] = 1

j=2
for i in v:
  vocab[i]=j
  j+=1
print(vocab)

defaultdict(<function <lambda> at 0x7f495107ac80>, {'<PAD>': 1, 'eat': 2, 'ad': 3, 'health4uandpets': 4, 'u': 5, 'upset': 6, 'food': 7, 'tonight': 8, 'man': 9, 'r': 10, '2y1zl': 11, 'also': 12, 'today': 13, 'lov': 14, 'kenich': 15, 'effect': 16, 'feel': 17, '50': 18, 'sav': 19, 'com': 20, 'itchy': 21, 'show': 22, 'best': 23, 'hin': 24, 'much': 25, 'switchfoot': 26, 'facebook': 27, 'whol': 28, 'are': 29, 'friend': 30, 'text': 31, 'bum': 32, 'might': 33, 'darealsunisakim': 34, 'cry': 35, 'guy': 36, 'girl': 37, 'bound': 38, 'lik': 39, 'thank': 40, 'sick': 41, 'david': 42, 'im': 43, 'hurt': 44, 'dc': 45, 'awww': 46, 'go': 47, 'many': 48, 'cant': 49, 'everyon': 50, 'meet': 51, 'real': 52, 'bal': 53, 'lovesbrooklyn2': 54, 'twitp': 55, 'result': 56, 'third': 57, 'behav': 58, 'http': 59, 'plu': 60, 'fir': 61, 'sunis': 62, 'see': 63, 'twit': 64, 'car': 65, 'nationwideclass': 66, 'upd': 67, 'cheap': 68, 'got': 69, 'body': 70, 'sweetheart': 71, 'should': 72, 'besty': 73, 'rest': 74, 'tim': 75, 't

In [ ]:
def transformWords(data):
  maxLen=0
  for i in range(len(data)):
    for j in range(len(data[i])):
      if (maxLen < len(data[i][j])):
        maxLen= len(data[i][j])

  for i in range(len(data)):
    for j in range(len(data[i])):
      test=[]
      for k in range(max):
        if (k >= len(data[i][j])):
          test.append(vocab['<PAD>'])
        else:
          test.append(vocab[data[i][j][k]])
      data[i][j]=np.array(test)
  return data

In [ ]:
data=transformWords(data)
datas=np.array([df.iloc[5:10][5],df.iloc[800005:800010][5]])
datas=processing(datas)
datas=transformWords(datas)

[[list(['kweside', 'whol', 'crew']) list(['nee', 'hug'])
  list(['lolt', 'hey', 'long', 'tim', 'see', 'ye', 'rain', 'bit', 'bit', 'lol', 'fin', 'thank'])
  list(['tatiana_k', 'nop']) list(['twitter', 'que', 'muer'])]
 [list(['productoffear', 'tel', 'burst', 'laugh', 'real', 'loud', 'thank', 'mak', 'com', 'sulk'])
  list(['r_keith_hill', 'than', 'respons', 'ihad', 'already', 'find', 'answ'])
  list(['keepinupwkr', 'jeal', 'hop', 'gre', 'tim', 'vega', 'lik', 'acm', 'lov', 'show'])
  list(['tommcf', 'ah', 'congr', 'mr', 'fletch', 'fin', 'join', 'twit'])
  list(['e4voip', 'respond', 'stupid', 'cat', 'help', 'typ', 'forg', 'er'])]]


In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(vocab),output_dim=2))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(1))

In [ ]:
data=np.concatenate([data[0],data[1]])
datas=np.concatenate([datas[0],datas[1]])

In [ ]:
print(data)
print(datas)

[array([26, 59, 55, 20, 11, 46, 32, 72, 69, 42, 65, 57, 84])
 array([ 6, 67, 27, 31, 33, 35, 56, 85, 13, 12, 82,  1,  1])
 array([15, 79, 48, 75, 53,  9, 19, 18, 74, 47, 38,  1,  1])
 array([28, 70, 17, 21, 39, 61,  1,  1,  1,  1,  1,  1,  1])
 array([66, 58, 80, 63,  1,  1,  1,  1,  1,  1,  1,  1,  1])
 array([14,  4,  5, 36, 10, 23,  1,  1,  1,  1,  1,  1,  1])
 array([43, 51, 83, 73,  8, 49, 81, 37, 76,  1,  1,  1,  1])
 array([34, 40, 64,  3, 62, 69, 51, 24, 22, 45, 29, 71,  1])
 array([41, 52, 68, 44, 25,  2, 52,  7, 60, 30, 77, 78,  1])
 array([54, 16, 50,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])]
[array([ 0, 28,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])
 array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
 array([ 0,  0,  0, 75, 63,  0,  0,  0,  0,  0,  0, 40,  1])
 array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
 array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
 array([ 0,  0,  0,  0, 52,  0, 40, 77, 20,  0,  1,  1,  1])
 array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
 array([ 0,  0

In [ ]:
y_train=np.concatenate([np.zeros((5,1)),np.ones((5,1))])
model.compile(loss='binary_crossentropy',optimizer="sgd")
model.fit(data,y_train,validation_data=(datas,y_train))

ValueError: ignored